# Inside Account B
This notebook self copies S3 objects (parquet files a.k.a features) created by Account B and redefines ACLs to enable access for Accounts A & C to use them. Use this notebook if this is a required pattern for your organization. This way feature groups/features created by Account B can be accessed directly by other accounts (A & C). This notebook is optional to run.

#### Imports 

In [1]:
import boto3

In [2]:
!aws --version

aws-cli/1.18.179 Python/3.6.10 Linux/4.14.200-116.320.amzn1.x86_64 botocore/1.19.19


#### Essentials

In [3]:
region = boto3.Session().region_name
s3_client = boto3.client('s3', region_name=region)
account_id = boto3.client('sts').get_caller_identity()['Account']
print(f'Account B = {account_id}')
bucket = 'sagemaker-feature-store-account-a'
feature_group_name = 'employees'

Account B = 149456328460


In [4]:
feature_group_s3_prefix = f'{account_id}/sagemaker/{region}/offline-store/{feature_group_name}/data'
feature_group_s3_prefix

'149456328460/sagemaker/us-east-1/offline-store/employees/data'

In [5]:
offline_store_contents = None
objects = s3_client.list_objects(Bucket=bucket, Prefix=feature_group_s3_prefix)
if 'Contents' in objects and len(objects['Contents']) > 1:
    offline_store_contents = objects['Contents']
offline_store_contents

[{'Key': '149456328460/sagemaker/us-east-1/offline-store/employees/data/year=2020/month=12/day=27/hour=05/20201227T055804Z_g0RfUU00NRUTPNzQ.parquet',
  'LastModified': datetime.datetime(2020, 12, 27, 6, 4, 34, tzinfo=tzlocal()),
  'ETag': '"56b640f74f7af0a8500626b697ae8a62"',
  'Size': 2137,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'arunprsh',
   'ID': '149b24f8987e48d549b9c2b494029c94d6c1e8b7b91092cad62ca7cd89aea747'}},
 {'Key': '149456328460/sagemaker/us-east-1/offline-store/employees/data/year=2020/month=12/day=27/hour=05/20201227T055804Z_sDCkKUEjSM6evhC4.parquet',
  'LastModified': datetime.datetime(2020, 12, 27, 6, 4, 34, tzinfo=tzlocal()),
  'ETag': '"e8082d7c64062858b4147311a0997c21"',
  'Size': 2081,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'arunprsh',
   'ID': '149b24f8987e48d549b9c2b494029c94d6c1e8b7b91092cad62ca7cd89aea747'}}]

In [6]:
can_id_a = "a52ce3999cdab5111cb19ca94abf5de5a69d62f34baa7d4422c630549fad3bd0"
can_id_b = "149b24f8987e48d549b9c2b494029c94d6c1e8b7b91092cad62ca7cd89aea747"
can_id_c = "768394a884ee2c604687e993ff8f4f5e6320bac8de2bba100ae7686a611b9260"

In [7]:
kms_id = "arn:aws:kms:us-east-1:892313895307:key/d3763b61-8d94-43bd-a3d6-4b4516ad28e7"

#### Redefine ACLs and self copy S3 objects to grant access to Account's A & C

In [8]:
for content in offline_store_contents:
    key = content['Key']
    print(f'Key = {key}')
    !aws s3api copy-object --copy-source {bucket}/{key} --key {key} --bucket {bucket} --server-side-encryption aws:kms  --ssekms-key-id {kms_id} && echo "[Self-Copy Succeeded!]"
    !aws s3api put-object-acl --bucket {bucket} --key {key} --grant-full-control id=$can_id_a,id=$can_id_b,id=$can_id_c && echo "[Redefine Object ACLs Succeeded!]"
     
    

Key = 149456328460/sagemaker/us-east-1/offline-store/employees/data/year=2020/month=12/day=27/hour=05/20201227T055804Z_g0RfUU00NRUTPNzQ.parquet
{
    "ServerSideEncryption": "aws:kms",
    "SSEKMSKeyId": "arn:aws:kms:us-east-1:892313895307:key/d3763b61-8d94-43bd-a3d6-4b4516ad28e7",
    "CopyObjectResult": {
        "ETag": "\"673b16948782325ad8fa1d9aa27d1c9f\"",
        "LastModified": "2020-12-27T06:05:45.000Z"
    }
}
[Self-Copy Succeeded!]
[Redefine Object ACLs Succeeded!]
Key = 149456328460/sagemaker/us-east-1/offline-store/employees/data/year=2020/month=12/day=27/hour=05/20201227T055804Z_sDCkKUEjSM6evhC4.parquet
{
    "ServerSideEncryption": "aws:kms",
    "SSEKMSKeyId": "arn:aws:kms:us-east-1:892313895307:key/d3763b61-8d94-43bd-a3d6-4b4516ad28e7",
    "CopyObjectResult": {
        "ETag": "\"ba070c86bcedfc828a3a7531197d2f03\"",
        "LastModified": "2020-12-27T06:05:48.000Z"
    }
}
[Self-Copy Succeeded!]
[Redefine Object ACLs Succeeded!]
